# Importing all the things

In [202]:
import numpy as np
import pandas as pd
import re
from urllib.request import urlopen
from bs4 import BeautifulSoup

%matplotlib inline

print('everything imported')


everything imported


## Part 1.   Getting the data using Beautiful soup

In [203]:
#creating URL
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
html = urlopen(url)

#making the soup
soup = BeautifulSoup(html, 'lxml')
type(soup)



bs4.BeautifulSoup

In [204]:
#making the soup
soup = BeautifulSoup(html, 'lxml')
type(soup)

bs4.BeautifulSoup

In [205]:
#getting the table we need as there are 2

table = soup.find('table', {'class':'wikitable'})

#Pulling the rows we need

for row in table:
    tr_rows = table.find_all('tr')
                        
#print(tr_rows[:10])
type(tr_rows)

TypeError: 'NoneType' object is not iterable

In [206]:
#pulling the cells and cleaning out the HTML

data_list = []

for row in tr_rows:
    td_rows = row.find_all('td')
    str_cells = str(td_rows)
    cleantext = BeautifulSoup(str_cells, "lxml").get_text()
    data_list.append(cleantext)
    

print('we have our data!')
    

we have our data!


## Now we will be creating our dataframe and cleaning up the data

In [207]:
df = pd.DataFrame(data_list)
df.head(10)

,0
0,[]
1,"[M1A\n, Not assigned\n, \n]"
2,"[M2A\n, Not assigned\n, \n]"
3,"[M3A\n, North York\n, Parkwoods\n]"
4,"[M4A\n, North York\n, Victoria Village\n]"
5,"[M5A\n, Downtown Toronto\n, Regent Park / Harb..."
6,"[M6A\n, North York\n, Lawrence Manor / Lawrenc..."
7,"[M7A\n, Downtown Toronto\n, Queen's Park / Ont..."
8,"[M8A\n, Not assigned\n, \n]"
9,"[M9A\n, Etobicoke\n, Islington Avenue\n]"


In [208]:
#Split up Dataframe
df = df[0].str.split(',', expand=True)
print(df.shape)
df.head(10)


(181, 4)


,0,1,2,3
0,[],None,None,None
1,[M1A\n,Not assigned\n,\n],None
2,[M2A\n,Not assigned\n,\n],None
3,[M3A\n,North York\n,Parkwoods\n],None
4,[M4A\n,North York\n,Victoria Village\n],None
5,[M5A\n,Downtown Toronto\n,Regent Park / Harbourfront\n],None
6,[M6A\n,North York\n,Lawrence Manor / Lawrence Heights\n],None
7,[M7A\n,Downtown Toronto\n,Queen's Park / Ontario Provincial Government\n],None
8,[M8A\n,Not assigned\n,\n],None
9,[M9A\n,Etobicoke\n,Islington Avenue\n],None


In [209]:
#clean up the [] and \n's

df[0] = df[0].str.strip('[')
df[0] = df[0].str.strip('\n')
df[2] = df[2].str.strip(']')
df[2] = df[2].str.strip('\n')
df[1] = df[1].str.strip('\n')


df.head(5)

,0,1,2,3
0,],None,None,None
1,M1A,Not assigned,,None
2,M2A,Not assigned,,None
3,M3A,North York,Parkwoods,None
4,M4A,North York,Victoria Village,None


In [210]:
# Dropping column 3 we dont need it
df = df.drop(df.columns[[3]], axis=1)
df.head(10)

,0,1,2
0,],None,None
1,M1A,Not assigned,
2,M2A,Not assigned,
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Regent Park / Harbourfront
6,M6A,North York,Lawrence Manor / Lawrence Heights
7,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government
8,M8A,Not assigned,
9,M9A,Etobicoke,Islington Avenue


In [211]:
#giving our columns some names and reindexing

df.columns = ['Postcode' , 'Borough', 'Neighborhood']
df.set_index('Postcode', inplace=True)
print(df.shape)
df.head(5)

(181, 2)


,Borough,Neighborhood
Postcode,,
],None,None
M1A,Not assigned,
M2A,Not assigned,
M3A,North York,Parkwoods
M4A,North York,Victoria Village


In [212]:
#Drop first row and not assigned postcodes

df.drop(']', inplace=True)

df = df[df['Borough'] != ' Not assigned']


df.head(10)


,Borough,Neighborhood
Postcode,,
M3A,North York,Parkwoods
M4A,North York,Victoria Village
M5A,Downtown Toronto,Regent Park / Harbourfront
M6A,North York,Lawrence Manor / Lawrence Heights
M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government
M9A,Etobicoke,Islington Avenue
M1B,Scarborough,Malvern / Rouge
M3B,North York,Don Mills
M4B,East York,Parkview Hill / Woodbine Gardens


In [213]:
#checking for non assigned Neighborhoods
df1 = df[df['Neighborhood'] == 'Not assigned']
print("none exist!")
df1.head()

none exist!


,Borough,Neighborhood
Postcode,,


In [214]:
#replacing the / with , in the Neighborhood column
df['Neighborhood'] = df['Neighborhood'].str.replace('/',',')
df.head(10)


,Borough,Neighborhood
Postcode,,
M3A,North York,Parkwoods
M4A,North York,Victoria Village
M5A,Downtown Toronto,"Regent Park , Harbourfront"
M6A,North York,"Lawrence Manor , Lawrence Heights"
M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government"
M9A,Etobicoke,Islington Avenue
M1B,Scarborough,"Malvern , Rouge"
M3B,North York,Don Mills
M4B,East York,"Parkview Hill , Woodbine Gardens"


## The Shape of our Cleaned up Dataframe

In [215]:
df.shape

(103, 2)

## PART 2 Trying to Geocode the Data 


Geocode is not responding looking at the docs it looks like a number have things have changed so using the .csv

In [242]:
pcode = pd.read_csv('https://cocl.us/Geospatial_data') 
pcode.head(5)


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [232]:
#indexing
pcode.set_index('Postal Code', inplace=True)
pcode.head()

,Latitude,Longitude
Postal Code,,
M1B,43.806686,-79.194353
M1C,43.784535,-79.160497
M1E,43.763573,-79.188711
M1G,43.770992,-79.216917
M1H,43.773136,-79.239476


In [241]:
#Merging the 2 data sets
df = pd.merge(df, pcode, left_index=True, right_index=True)
df.head(10)

,Borough,Neighborhood,Latitude,Longitude
M3A,North York,Parkwoods,43.753259,-79.329656
M4A,North York,Victoria Village,43.725882,-79.315572
M5A,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636
M6A,North York,"Lawrence Manor , Lawrence Heights",43.718518,-79.464763
M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government",43.662301,-79.389494
M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
M1B,Scarborough,"Malvern , Rouge",43.806686,-79.194353
M3B,North York,Don Mills,43.745906,-79.352188
M4B,East York,"Parkview Hill , Woodbine Gardens",43.706397,-79.309937
M5B,Downtown Toronto,Garden District,43.657162,-79.378937


## Part 3 Clustering

In [246]:
# Just going to Use Toronto Data to keep this simple

toronto = df[df['Borough'].str.contains("Toronto")]
toronto.head(5)



,Borough,Neighborhood,Latitude,Longitude
M5A,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636
M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government",43.662301,-79.389494
M5B,Downtown Toronto,Garden District,43.657162,-79.378937
M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
M4E,East Toronto,The Beaches,43.676357,-79.293031


Get Environment set up for next part

In [250]:
!pip install geopy

     |████████████████████████████████| 104 kB 1.9 MB/s eta 0:00:01


In [252]:
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium # map rendering library
from geopy.geocoders import Nominatim
import json


Plotting the raw data on the map

In [253]:
# Get Coordinates for Toronto
address = 'Toronto , Ontario'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of Toronto are 43.6534817, -79.3839347.


In [254]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto['Latitude'], toronto['Longitude'], toronto['Borough'], toronto['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

# Foursquaring


In [258]:
#Function for extracting the venue data


def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [259]:
#use our toronto data to do the venues
toronto_venues = getNearbyVenues(names=toronto['Neighborhood'],
                                   latitudes=toronto['Latitude'],
                                   longitudes=toronto['Longitude']
                                  )

toronto_venues.head()



 Regent Park , Harbourfront
 Queen's Park , Ontario Provincial Government
 Garden District
 St. James Town
 The Beaches
 Berczy Park
 Central Bay Street
 Christie
 Richmond , Adelaide , King
 Dufferin , Dovercourt Village
 Harbourfront East , Union Station , Toronto Islands
 Little Portugal , Trinity
 The Danforth West , Riverdale
 Toronto Dominion Centre , Design Exchange
 Brockton , Parkdale Village , Exhibition Place
 India Bazaar , The Beaches West
 Commerce Court , Victoria Hotel
 Studio District
 Lawrence Park
 Roselawn
 Davisville North
 Forest Hill North & West
 High Park , The Junction South
 North Toronto West
 The Annex , North Midtown , Yorkville
 Parkdale , Roncesvalles
 Davisville
 University of Toronto , Harbord
 Runnymede , Swansea
 Moore Park , Summerhill East
 Kensington Market , Chinatown , Grange Park
 Summerhill West , Rathnelly , South Hill , Forest Hill SE , Deer Park
 CN Tower , King and Spadina , Railway Lands , Harbourfront West , Bathurst
 Quay , South Niagar

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park , Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park , Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park , Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,"Regent Park , Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,"Regent Park , Harbourfront",43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot


In [260]:
toronto_venues.shape

(866, 7)

venues per neighborhood

In [261]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,30,30,30,30,30,30
"Brockton , Parkdale Village , Exhibition Place",23,23,23,23,23,23
Business reply mail Processing CentrE,18,18,18,18,18,18
"CN Tower , King and Spadina , Railway Lands , Harbourfront West , Bathurst\n Quay , South Niagara , Island airport",16,16,16,16,16,16
Central Bay Street,30,30,30,30,30,30
Christie,18,18,18,18,18,18
Church and Wellesley,30,30,30,30,30,30
"Commerce Court , Victoria Hotel",30,30,30,30,30,30
Davisville,30,30,30,30,30,30


In [262]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 188 uniques categories.


Analyse the Neighborhood

In [313]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")


In [316]:
toronto_onehot['Neighborhood1'] = toronto_venues['Neighborhood']
# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

In [317]:
toronto_onehot.head()

,Neighborhood1,Yoga Studio,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar
0,"Regent Park , Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Regent Park , Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Regent Park , Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Regent Park , Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Regent Park , Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Grouping and Mean Calcs

In [318]:
#calc mean
toronto_grouped = toronto_onehot.groupby('Neighborhood1').mean().reset_index()
toronto_grouped.head()

,Neighborhood1,Yoga Studio,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar
0,Berczy Park,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.0,0.0,...,0.033333,0.0,0.0,0.0,0.0,0.0,0.033333,0.0,0.0,0.0
1,"Brockton , Parkdale Village , Exhibition Place",0.043478,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
2,Business reply mail Processing CentrE,0.055556,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
3,"CN Tower , King and Spadina , Railway Lands ,...",0.000000,0.0625,0.0625,0.0625,0.125,0.1875,0.125,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
4,Central Bay Street,0.033333,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.033333,0.0,0.0,0.0


Top Venues per Neighborhood

In [319]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood1']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood1'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

---- Berczy Park----
                venue  freq
0         Coffee Shop  0.10
1              Bakery  0.07
2                Café  0.07
3            Beer Bar  0.07
4  Seafood Restaurant  0.07


---- Brockton , Parkdale Village , Exhibition Place----
            venue  freq
0            Café  0.13
1     Coffee Shop  0.09
2  Breakfast Spot  0.09
3     Yoga Studio  0.04
4          Bakery  0.04


---- Business reply mail Processing CentrE----
                  venue  freq
0           Yoga Studio  0.06
1  Fast Food Restaurant  0.06
2                  Park  0.06
3           Pizza Place  0.06
4      Recording Studio  0.06


---- CN Tower , King and Spadina , Railway Lands , Harbourfront West , Bathurst
 Quay , South Niagara , Island airport----
              venue  freq
0   Airport Service  0.19
1    Airport Lounge  0.12
2  Airport Terminal  0.12
3           Airport  0.06
4             Plane  0.06


---- Central Bay Street----
                       venue  freq
0                Coffee Shop  0.23

In [320]:
#Function for Putting into a Dataframe 

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [325]:
#creating the Dataframe

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood1']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood1'] = toronto_grouped['Neighborhood1']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood1,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Seafood Restaurant,Bakery,Café,Beer Bar,Farmers Market,Cocktail Bar,Fountain,Park,Bistro
1,"Brockton , Parkdale Village , Exhibition Place",Café,Breakfast Spot,Coffee Shop,Yoga Studio,Performing Arts Venue,Bar,Restaurant,Burrito Place,Pet Store,Furniture / Home Store
2,Business reply mail Processing CentrE,Yoga Studio,Fast Food Restaurant,Recording Studio,Brewery,Skate Park,Smoke Shop,Spa,Burrito Place,Pizza Place,Gym / Fitness Center
3,"CN Tower , King and Spadina , Railway Lands ,...",Airport Service,Airport Lounge,Airport Terminal,Harbor / Marina,Airport,Airport Food Court,Airport Gate,Boutique,Sculpture Garden,Bar
4,Central Bay Street,Coffee Shop,Italian Restaurant,Café,Spa,Gastropub,Park,Modern European Restaurant,Miscellaneous Shop,Middle Eastern Restaurant,Japanese Restaurant


# Clustering

In [348]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood1', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:30] 

array([0, 0, 2, 2, 0, 0, 2, 0, 0, 2, 0, 0, 2, 0, 0, 2, 2, 0, 4, 2, 3, 2,
       2, 0, 0, 0, 4, 1, 0, 0], dtype=int32)

In [349]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)



ValueError: cannot insert Cluster Labels, already exists

In [350]:
toronto_merged = toronto

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood1'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
M5A,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636,0,Coffee Shop,Bakery,Park,Café,Mexican Restaurant,Breakfast Spot,Farmers Market,Dessert Shop,Distribution Center,Spa
M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government",43.662301,-79.389494,0,Coffee Shop,Diner,Yoga Studio,Burger Joint,Beer Bar,Distribution Center,Discount Store,Sandwich Place,Boutique,Burrito Place
M5B,Downtown Toronto,Garden District,43.657162,-79.378937,0,Clothing Store,Coffee Shop,Café,Burger Joint,Diner,Sandwich Place,Restaurant,Hotel,Ramen Restaurant,Plaza
M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,0,Café,Gastropub,Hotel,Coffee Shop,Japanese Restaurant,Restaurant,Hostel,Ice Cream Shop,Diner,Beer Bar
M4E,East Toronto,The Beaches,43.676357,-79.293031,2,Trail,Pub,Health Food Store,Wine Bar,Coworking Space,Discount Store,Diner,Dessert Shop,Department Store,Deli / Bodega


In [351]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# Cluster's

In [353]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
M5A,"Regent Park , Harbourfront",Coffee Shop,Bakery,Park,Café,Mexican Restaurant,Breakfast Spot,Farmers Market,Dessert Shop,Distribution Center,Spa
M7A,"Queen's Park , Ontario Provincial Government",Coffee Shop,Diner,Yoga Studio,Burger Joint,Beer Bar,Distribution Center,Discount Store,Sandwich Place,Boutique,Burrito Place
M5B,Garden District,Clothing Store,Coffee Shop,Café,Burger Joint,Diner,Sandwich Place,Restaurant,Hotel,Ramen Restaurant,Plaza
M5C,St. James Town,Café,Gastropub,Hotel,Coffee Shop,Japanese Restaurant,Restaurant,Hostel,Ice Cream Shop,Diner,Beer Bar
M5E,Berczy Park,Coffee Shop,Seafood Restaurant,Bakery,Café,Beer Bar,Farmers Market,Cocktail Bar,Fountain,Park,Bistro
M5G,Central Bay Street,Coffee Shop,Italian Restaurant,Café,Spa,Gastropub,Park,Modern European Restaurant,Miscellaneous Shop,Middle Eastern Restaurant,Japanese Restaurant
M6G,Christie,Grocery Store,Café,Park,Coffee Shop,Restaurant,Baby Store,Candy Store,Italian Restaurant,Athletics & Sports,Diner
M5H,"Richmond , Adelaide , King",Café,Asian Restaurant,Pizza Place,Coffee Shop,Gastropub,Restaurant,Gym / Fitness Center,Smoke Shop,Bar,Plaza
M6H,"Dufferin , Dovercourt Village",Bakery,Pharmacy,Café,Supermarket,Bank,Bar,Grocery Store,Gym / Fitness Center,Park,Brewery
M5J,"Harbourfront East , Union Station , Toronto I...",Plaza,Park,Café,Hotel,Ice Cream Shop,Skating Rink,Dessert Shop,Bistro,Dance Studio,Salad Place


In [354]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
M5N,Roselawn,Garden,Wine Bar,Cosmetics Shop,Discount Store,Diner,Dessert Shop,Department Store,Deli / Bodega,Dance Studio,Cuban Restaurant


In [355]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
M4E,The Beaches,Trail,Pub,Health Food Store,Wine Bar,Coworking Space,Discount Store,Diner,Dessert Shop,Department Store,Deli / Bodega
M6J,"Little Portugal , Trinity",Bar,Asian Restaurant,Vietnamese Restaurant,Wine Bar,French Restaurant,New American Restaurant,Men's Store,Korean Restaurant,Japanese Restaurant,Italian Restaurant
M4K,"The Danforth West , Riverdale",Greek Restaurant,Ice Cream Shop,Italian Restaurant,Yoga Studio,Fruit & Vegetable Store,Restaurant,Brewery,Bubble Tea Shop,Pub,Café
M4L,"India Bazaar , The Beaches West",Fast Food Restaurant,Ice Cream Shop,Pet Store,Board Shop,Restaurant,Italian Restaurant,Brewery,Pub,Burrito Place,Steakhouse
M4P,Davisville North,Convenience Store,Park,Department Store,Sandwich Place,Breakfast Spot,Hotel,Food & Drink Shop,Gym,Coworking Space,Dessert Shop
M5P,Forest Hill North & West,Trail,Jewelry Store,Mexican Restaurant,Sushi Restaurant,Wine Bar,Coworking Space,Diner,Dessert Shop,Department Store,Deli / Bodega
M6P,"High Park , The Junction South",Thai Restaurant,Grocery Store,Café,Mexican Restaurant,Bar,Fast Food Restaurant,Diner,Italian Restaurant,Convenience Store,Bakery
M4R,North Toronto West,Clothing Store,Coffee Shop,Miscellaneous Shop,Salon / Barbershop,Restaurant,Rental Car Location,Café,Pet Store,Park,Yoga Studio
M6R,"Parkdale , Roncesvalles",Gift Shop,Coffee Shop,Bookstore,Dog Run,Breakfast Spot,Dessert Shop,Eastern European Restaurant,Bar,Restaurant,Cuban Restaurant
M4V,"Summerhill West , Rathnelly , South Hill , Fo...",Coffee Shop,Pub,Sushi Restaurant,Liquor Store,Park,Light Rail Station,Pizza Place,Restaurant,Bank,Sports Bar


In [356]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
M4T,"Moore Park , Summerhill East",Playground,Tennis Court,Wine Bar,Convenience Store,Diner,Dessert Shop,Department Store,Deli / Bodega,Dance Studio,Cuban Restaurant


In [357]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
M4N,Lawrence Park,Bus Line,Park,Swim School,Cosmetics Shop,Diner,Dessert Shop,Department Store,Deli / Bodega,Dance Studio,Cuban Restaurant
M4W,Rosedale,Park,Trail,Playground,Wine Bar,Convenience Store,Diner,Dessert Shop,Department Store,Deli / Bodega,Dance Studio
